In [0]:
! pip install scikit-plot

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import xgboost as xgb
from sklearn import tree, neighbors, datasets, linear_model, svm, naive_bayes, ensemble
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import cross_validate,GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.utils.multiclass import unique_labels
import scikitplot as skplt
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import lightgbm as lgbm
from sklearn import metrics
from sklearn import model_selection
import warnings
warnings.filterwarnings('ignore')

In [0]:
from google.colab import files
uploaded = files.upload()

Saving test_final.csv to test_final.csv
Saving train_final.csv to train_final.csv


In [0]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_dir = '/content/drive/Shared drives/Predictive Competition/recruit-restaurant-visitor-forecasting/'

In [0]:
submission = pd.read_csv(data_dir + 'sample_submission.csv')
submission['air_store_id'] = submission['id'].str.slice(0, 20)
submission['visit_date'] = submission['id'].str.slice(21)
submission['is_test'] = True
submission['visitors'] = np.nan
submission['test_number'] = range(len(submission))

submission.head()

,id,visitors,air_store_id,visit_date,is_test,test_number
0,air_00a91d42b08b08d9_2017-04-23,NaN,air_00a91d42b08b08d9,2017-04-23,True,0
1,air_00a91d42b08b08d9_2017-04-24,NaN,air_00a91d42b08b08d9,2017-04-24,True,1
2,air_00a91d42b08b08d9_2017-04-25,NaN,air_00a91d42b08b08d9,2017-04-25,True,2
3,air_00a91d42b08b08d9_2017-04-26,NaN,air_00a91d42b08b08d9,2017-04-26,True,3
4,air_00a91d42b08b08d9_2017-04-27,NaN,air_00a91d42b08b08d9,2017-04-27,True,4


In [0]:
train = pd.read_csv(data_dir + "sam_train_final.csv")
test = pd.read_csv(data_dir + "sam_test_final.csv")

In [0]:
train.head()

,Unnamed: 0,Unnamed: 0.1,visit_date,air_store_id,geo1,population,density,Food_Type,is_holiday,next_day,prev_day,visitors_x,season,week,daysToPrev25th,prev_visitors,year,month_y,Dining_yes,day,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,reserve_visitors,days_diff,latitude_y,longitude_y,day_of_week_y,Consecutive_holidays,Weekend,summer_yes,is_outlier,visitors_capped,visitors_capped_log1p
0,0,0,2016-01-13,air_ba937bf13d40fb24,Tokyo-to,2.7,11.9,Bars,0,0.0,0.0,25.0,winter,2,19,NaN,2016,1,0,13,7,23.843750,25.0,57,64,-1.0,-1.0,35.658068,139.751599,Wednesday,0.0,0,0,False,35.0,3.583519
1,1,1,2016-01-14,air_ba937bf13d40fb24,Tokyo-to,2.7,11.9,Bars,0,0.0,0.0,32.0,winter,2,20,NaN,2016,1,0,14,2,20.292308,21.0,54,65,-1.0,-1.0,35.658068,139.751599,Thursday,0.0,0,0,False,9.0,2.302585
2,2,2,2016-01-15,air_ba937bf13d40fb24,Tokyo-to,2.7,11.9,Bars,0,0.0,0.0,29.0,winter,2,21,NaN,2016,1,0,15,4,34.738462,35.0,61,65,-1.0,-1.0,35.658068,139.751599,Friday,0.0,0,0,False,20.0,3.044522
3,3,3,2016-01-16,air_ba937bf13d40fb24,Tokyo-to,2.7,11.9,Bars,0,0.0,0.0,22.0,winter,2,22,NaN,2016,1,0,16,6,27.651515,27.0,53,66,-1.0,-1.0,35.658068,139.751599,Saturday,0.0,1,0,False,25.0,3.258097
4,4,4,2016-01-18,air_ba937bf13d40fb24,Tokyo-to,2.7,11.9,Bars,0,0.0,0.0,6.0,winter,3,24,NaN,2016,1,0,18,2,13.754386,12.0,34,57,-1.0,-1.0,35.658068,139.751599,Monday,0.0,0,0,False,29.0,3.401197


In [0]:
test.head()

,Unnamed: 0,Unnamed: 0.1,visit_date,air_store_id,geo1,density,population,Food_Type,is_holiday,next_day,prev_day,season,Dining_yes,week,daysToPrev25th,prev_visitors,year,month_y,day,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,reserve_visitors,days_diff,latitude_y,longitude_y,day_of_week_y,Consecutive_holidays,Weekend,summer_yes
0,0,0,2017-04-23,air_00a91d42b08b08d9,Tokyo-to,11.9,2.7,European,0,0.0,0.0,spring,1,16,29,7.0,2017,4,23,2.0,2.000000,2.0,2.0,1.0,-1.0,-1.0,35.694003,139.753595,Sunday,0.0,1,0
1,1,1,2017-04-24,air_00a91d42b08b08d9,Tokyo-to,11.9,2.7,European,0,0.0,0.0,spring,1,17,30,2.0,2017,4,24,1.0,22.457143,19.0,47.0,35.0,-1.0,-1.0,35.694003,139.753595,Monday,0.0,0,0
2,2,2,2017-04-25,air_00a91d42b08b08d9,Tokyo-to,11.9,2.7,European,0,0.0,0.0,spring,1,17,0,2.0,2017,4,25,1.0,24.350000,24.5,43.0,40.0,-1.0,-1.0,35.694003,139.753595,Tuesday,0.0,0,0
3,3,3,2017-04-26,air_00a91d42b08b08d9,Tokyo-to,11.9,2.7,European,0,0.0,0.0,spring,1,17,1,9.0,2017,4,26,15.0,28.125000,28.0,52.0,40.0,-1.0,-1.0,35.694003,139.753595,Wednesday,0.0,0,0
4,4,4,2017-04-27,air_00a91d42b08b08d9,Tokyo-to,11.9,2.7,European,0,0.0,0.0,spring,1,17,2,22.0,2017,4,27,15.0,29.868421,30.0,47.0,38.0,-1.0,-1.0,35.694003,139.753595,Thursday,0.0,0,0


In [0]:
# train['visitors_log1p'] = np.log1p(train['visitors_x'])
train.index = train['visit_date']
train = train.drop(columns = ['visit_date', 'Unnamed: 0','Unnamed: 0.1', 'air_store_id', 'reserve_visitors', 'days_diff','geo1','visitors_capped','is_outlier'])
train.head()

,population,density,Food_Type,is_holiday,next_day,prev_day,visitors_x,season,week,daysToPrev25th,prev_visitors,year,month_y,Dining_yes,day,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,latitude_y,longitude_y,day_of_week_y,Consecutive_holidays,Weekend,summer_yes,visitors_capped_log1p
visit_date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-13,2.7,11.9,Bars,0,0.0,0.0,25.0,winter,2,19,NaN,2016,1,0,13,7,23.843750,25.0,57,64,35.658068,139.751599,Wednesday,0.0,0,0,3.583519
2016-01-14,2.7,11.9,Bars,0,0.0,0.0,32.0,winter,2,20,NaN,2016,1,0,14,2,20.292308,21.0,54,65,35.658068,139.751599,Thursday,0.0,0,0,2.302585
2016-01-15,2.7,11.9,Bars,0,0.0,0.0,29.0,winter,2,21,NaN,2016,1,0,15,4,34.738462,35.0,61,65,35.658068,139.751599,Friday,0.0,0,0,3.044522
2016-01-16,2.7,11.9,Bars,0,0.0,0.0,22.0,winter,2,22,NaN,2016,1,0,16,6,27.651515,27.0,53,66,35.658068,139.751599,Saturday,0.0,1,0,3.258097
2016-01-18,2.7,11.9,Bars,0,0.0,0.0,6.0,winter,3,24,NaN,2016,1,0,18,2,13.754386,12.0,34,57,35.658068,139.751599,Monday,0.0,0,0,3.401197


In [0]:
test.index = test['visit_date']
test = test.drop(columns = ['visit_date', 'Unnamed: 0','Unnamed: 0.1','air_store_id', 'reserve_visitors', 'days_diff','geo1'])
test.head()

,density,population,Food_Type,is_holiday,next_day,prev_day,season,Dining_yes,week,daysToPrev25th,prev_visitors,year,month_y,day,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,latitude_y,longitude_y,day_of_week_y,Consecutive_holidays,Weekend,summer_yes
visit_date,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-23,11.9,2.7,European,0,0.0,0.0,spring,1,16,29,7.0,2017,4,23,2.0,2.000000,2.0,2.0,1.0,35.694003,139.753595,Sunday,0.0,1,0
2017-04-24,11.9,2.7,European,0,0.0,0.0,spring,1,17,30,2.0,2017,4,24,1.0,22.457143,19.0,47.0,35.0,35.694003,139.753595,Monday,0.0,0,0
2017-04-25,11.9,2.7,European,0,0.0,0.0,spring,1,17,0,2.0,2017,4,25,1.0,24.350000,24.5,43.0,40.0,35.694003,139.753595,Tuesday,0.0,0,0
2017-04-26,11.9,2.7,European,0,0.0,0.0,spring,1,17,1,9.0,2017,4,26,15.0,28.125000,28.0,52.0,40.0,35.694003,139.753595,Wednesday,0.0,0,0
2017-04-27,11.9,2.7,European,0,0.0,0.0,spring,1,17,2,22.0,2017,4,27,15.0,29.868421,30.0,47.0,38.0,35.694003,139.753595,Thursday,0.0,0,0


In [0]:
train_x = train.drop(columns = ['visitors_x','visitors_capped_log1p'])
# train_y = train[['visitors_log1p']]
train_y = train[['visitors_capped_log1p']]

In [0]:
len(train_x.columns),len(train_y.columns)

(42, 1)

In [0]:
train_x = pd.get_dummies(train_x, columns=['season', 'day_of_week_y', 'Food_Type'])
test_x = pd.get_dummies(test, columns=['season', 'day_of_week_y', 'Food_Type'])

In [0]:
len(test_x.columns)

42

In [0]:
test_x['season_autumn'] = 0
test_x['season_winter'] = 0
test_x['season_summer'] = 0

In [0]:
train_x.columns

Index(['population', 'density', 'is_holiday', 'next_day', 'prev_day', 'week',
       'daysToPrev25th', 'prev_visitors', 'year', 'month_y', 'Dining_yes',
       'day', 'min_visitors', 'mean_visitors', 'median_visitors',
       'max_visitors', 'count_observations', 'latitude_y', 'longitude_y',
       'Consecutive_holidays', 'Weekend', 'summer_yes', 'is_outlier',
       'visitors_capped', 'visitors_capped_log1p', 'season_autumn',
       'season_spring', 'season_summer', 'season_winter',
       'day_of_week_y_Friday', 'day_of_week_y_Monday',
       'day_of_week_y_Saturday', 'day_of_week_y_Sunday',
       'day_of_week_y_Thursday', 'day_of_week_y_Tuesday',
       'day_of_week_y_Wednesday', 'Food_Type_Asian', 'Food_Type_Bars',
       'Food_Type_Cafe', 'Food_Type_European',
       'Food_Type_International cuisine', 'Food_Type_Izakaya',
       'Food_Type_Japanese', 'Food_Type_Korean', 'Food_Type_Western/Create'],
      dtype='object')

In [0]:
test_x = test_x[['population', 'is_holiday', 'next_day', 'prev_day', 'week',
       'daysToPrev25th', 'prev_visitors', 'year', 'month_y', 'day',
       'min_visitors', 'mean_visitors', 'median_visitors', 'max_visitors',
       'count_observations', 'latitude_y', 'longitude_y',
       'Consecutive_holidays', 'season_autumn', 'season_spring',
       'season_summer', 'season_winter', 'day_of_week_y_Friday',
       'day_of_week_y_Monday', 'day_of_week_y_Saturday',
       'day_of_week_y_Sunday', 'day_of_week_y_Thursday',
       'day_of_week_y_Tuesday', 'day_of_week_y_Wednesday', 'Food_Type_Asian',
       'Food_Type_Bars', 'Food_Type_Cafe', 'Food_Type_European',
       'Food_Type_International cuisine', 'Food_Type_Izakaya',
       'Food_Type_Japanese', 'Food_Type_Korean', 'Food_Type_Western/Create']]

In [0]:
test_x.columns

Index(['population', 'is_holiday', 'next_day', 'prev_day', 'week',
       'daysToPrev25th', 'prev_visitors', 'year', 'month_y', 'day',
       'min_visitors', 'mean_visitors', 'median_visitors', 'max_visitors',
       'count_observations', 'latitude_y', 'longitude_y',
       'Consecutive_holidays', 'season_autumn', 'season_spring',
       'season_summer', 'season_winter', 'day_of_week_y_Friday',
       'day_of_week_y_Monday', 'day_of_week_y_Saturday',
       'day_of_week_y_Sunday', 'day_of_week_y_Thursday',
       'day_of_week_y_Tuesday', 'day_of_week_y_Wednesday', 'Food_Type_Asian',
       'Food_Type_Bars', 'Food_Type_Cafe', 'Food_Type_European',
       'Food_Type_International cuisine', 'Food_Type_Izakaya',
       'Food_Type_Japanese', 'Food_Type_Korean', 'Food_Type_Western/Create'],
      dtype='object')

In [0]:
#LGBM
np.random.seed(42)

model = lgbm.LGBMRegressor(
    objective='regression',
    max_depth=5,
    num_leaves=5 ** 2 - 1,
    learning_rate=0.007,
    n_estimators=30000,
    min_child_samples=80,
    subsample=0.8,
    colsample_bytree=1,
    reg_alpha=0,
    reg_lambda=0,
    random_state=np.random.randint(10e6)
)

n_splits = 8
cv = model_selection.KFold(n_splits=n_splits, shuffle=True, random_state=42)

val_scores = [0] * n_splits

sub = submission['id'].to_frame()
sub['visitors'] = 0

feature_importances = pd.DataFrame(index=train_x.columns)

for i, (fit_idx, val_idx) in enumerate(cv.split(train_x, train_y)):
    
    X_fit = train_x.iloc[fit_idx]
    y_fit = train_y.iloc[fit_idx]
    X_val = train_x.iloc[val_idx]
    y_val = train_y.iloc[val_idx]
    
    model.fit(
        X_fit,
        y_fit,
        eval_set=[(X_fit, y_fit), (X_val, y_val)],
        eval_names=('fit', 'val'),
        eval_metric='l2',
        early_stopping_rounds=200,
        feature_name=X_fit.columns.tolist(),
        verbose=False
    )
    
    val_scores[i] = np.sqrt(model.best_score_['val']['l2'])
    sub['visitors'] += model.predict(test_x, num_iteration=model.best_iteration_)
    feature_importances[i] = model.feature_importances_
    
    print('Fold {} RMSLE: {:.5f}'.format(i+1, val_scores[i]))
    
sub['visitors'] /= n_splits
sub['visitors'] = np.expm1(sub['visitors'])

val_mean = np.mean(val_scores)
val_std = np.std(val_scores)

print('Local RMSLE: {:.5f} (±{:.5f})'.format(val_mean, val_std))

ValueError: ignored

In [0]:
feature_importances.sort_values(0, ascending=False)

,0,1,2,3,4,5,6,7
prev_visitors,33487,16564,26615,24329,14396,20890,34392,27587
week,32048,17712,25184,24885,17254,22975,32059,25204
max_visitors,25635,14478,21413,18277,13231,16577,24974,20877
mean_visitors,23467,13446,18287,17847,13182,16083,24472,19790
median_visitors,22254,13710,17371,16871,12192,14946,20518,17603
count_observations,19178,10186,14743,13926,9553,13372,19285,15086
day,17556,9427,13284,13194,9900,12635,16371,13230
latitude_y,17167,8772,14552,12345,8361,11982,16824,13436
longitude_y,16006,8890,13461,12281,8471,10572,14320,13382
daysToPrev25th,14643,8362,11613,11897,7946,10134,14761,12345


In [0]:


from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
sub.to_csv('lgbm_11_29.csv', index = False)
!cp lgbm_11_29.csv drive/My\ Drive/